<a href="https://colab.research.google.com/github/aykb/Group-project-hand-in-/blob/main/gustave_challenge2_fairness_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge 2 – Data Quality, Fairness & SQL Sampling with AIF360

In this challenge you will:

1. Evaluate **data quality** and basic statistics of a socio-economic dataset.
2. Use the **IBM AI Fairness 360 (AIF360)** toolkit to detect bias and create a fairer dataset.
3. Implement **fair sampling** strategies in SQL when building a training sample for an AI model.

We will work with the **Adult Census Income** dataset (UCI Machine Learning Repository), which is bundled with AIF360.

- UCI dataset page: https://archive.ics.uci.edu/dataset/2/adult

👉 The ideas and metrics you develop here will be **reused in Challenge 3** to think about fairness in **regression** models for energy consumption.



In [ ]:
!pip install aif360

In [ ]:
# 🔧 Setup
# In Google Colab, install AIF360 once (this may take a couple of minutes):
!pip install -q aif360 pandas numpy matplotlib scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from aif360.datasets import AdultDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
plt.rcParams['figure.figsize'] = (10, 4)



## 1. Load the Adult dataset & assess data quality

1. Use `AdultDataset()` from AIF360 to load the data.  
2. Convert it to a pandas DataFrame using `convert_to_dataframe()`.  
3. Inspect:
   - Shape and column names
   - Missing values
   - Basic descriptive statistics

Treat this as a **data-quality check** step for an AI system that will decide who is eligible for a smart-city **energy-efficiency support programme**.



In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)


{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

In [ ]:
import pandas as pd
import numpy as np

# On combine features et targets dans un seul DataFrame pour l'analyse
df = pd.concat([X, y], axis=1)

# --- Data Quality Check ---
print(f"Format initial : {df.shape}")

# 1. Gestion des valeurs manquantes
# Le repo UCI utilise parfois '?' pour les valeurs manquantes
df = df.replace('?', np.nan)
print("\nValeurs manquantes par colonne :")
print(df.isnull().sum())

# Stratégie : On supprime les lignes avec des valeurs manquantes pour la qualité de l'IA
df_clean = df.dropna()

# 2. Encodage binaire pour AIF360
# Sexe : Male = 1, Female = 0
df_clean['sex'] = df_clean['sex'].map({'Male': 1, 'Female': 0})

# Revenu (Target) : >50K = 1, <=50K = 0
# Attention : selon la version, il peut y avoir un point à la fin (ex: '>50K.')
df_clean['income'] = df_clean['income'].str.replace('.', '', regex=False)
df_clean['income'] = df_clean['income'].map({'>50K': 1, '<=50K': 0})

print(f"\nFormat après nettoyage : {df_clean.shape}")
display(df_clean.head())

Format initial : (48842, 15)

Valeurs manquantes par colonne :
age                  0
workclass         2799
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
income               0
dtype: int64

Format après nettoyage : (45222, 15)


/tmp/ipython-input-856781916.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['sex'] = df_clean['sex'].map({'Male': 1, 'Female': 0})
/tmp/ipython-input-856781916.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['income'] = df_clean['income'].str.replace('.', '', regex=False)
/tmp/ipython-input-856781916.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Cuba,0


In [ ]:
# On définit les paramètres du dataset
# On ne garde que les colonnes numériques pour cet exercice simplifié
# (AIF360 nécessite des features numériques pour calculer les métriques)
df_numeric = pd.get_dummies(df_clean)

# Création de l'objet AIF360
dataset_aif = BinaryLabelDataset(
    df=df_numeric,
    label_names=['income'],
    favorable_label=1,
    unfavorable_label=0,
    protected_attribute_names=['sex']
)

print("Dataset prêt pour l'analyse de fairness avec AIF360.")

Dataset prêt pour l'analyse de fairness avec AIF360.


In [ ]:
import pandas as pd
import numpy as np

# NOTE: La classe AdultDataset() échoue car elle ne trouve pas les fichiers bruts.
# Cependant, le dataset a déjà été chargé et nettoyé en 'df_clean' dans les cellules précédentes.
# Nous utiliserons 'dataset_aif' (créé à partir de 'df_numeric', qui est lui-même basé sur 'df_clean')
# pour les métriques AIF360, et 'df_clean' pour la partie SQL.

print("Nous allons maintenant utiliser le DataFrame 'df_clean' et le BinaryLabelDataset 'dataset_aif' déjà préparés.")

# AIF360 encodes 'sex' as 1.0 for Male, 0.0 for Female.
# We can infer this from the metadata if adult_ds was successfully loaded, but since it failed,
# we rely on the common encoding for AdultDataset or what was done in df_clean.
# In df_clean, Male=1, Female=0, which matches AIF360's default.
# We assume the 'sex' column in df_numeric (used to create dataset_aif) is also 0/1.

# Define privileged and unprivileged groups based on the encoding in df_clean/dataset_aif
priv = [{'sex': 1}]   # Male (assuming 1 for Male in df_clean and dataset_aif)
unpriv = [{'sex': 0}] # Female (assuming 0 for Female in df_clean and dataset_aif)

# Now, directly use dataset_aif to create the metric object
# (dataset_aif was successfully created in cell Y9b-yY2rH4x5)
metric_orig = BinaryLabelDatasetMetric(
    dataset_aif,
    unprivileged_groups=unpriv,
    privileged_groups=priv
)

# Compute and print statistical_parity_difference and disparate_impact
spd = metric_orig.statistical_parity_difference()
di = metric_orig.disparate_impact()

print("\n--- Métriques d'Équité (Dataset Original via dataset_aif) ---")
print(f"Statistical Parity Difference (SPD) : {spd:.4f}")
print(f"Disparate Impact (DI)               : {di:.4f}")


Nous allons maintenant utiliser le DataFrame 'df_clean' et le BinaryLabelDataset 'dataset_aif' déjà préparés.

--- Métriques d'Équité (Dataset Original via dataset_aif) ---
Statistical Parity Difference (SPD) : -0.1989
Disparate Impact (DI)               : 0.3635


## 2. Measure bias with AIF360

We consider **sex** as a protected attribute and the income label (`income-per-year`) as the outcome.

Use AIF360 to:

1. Define:
   - Privileged group: `sex = Male`
   - Unprivileged group: `sex = Female`
2. Build a `BinaryLabelDatasetMetric` on the original dataset.
3. Compute at least:
   - `statistical_parity_difference`
   - `disparate_impact`
4. Interpret the results briefly in a markdown cell.

👉 Keep in mind: in **Challenge 3** you will design analogous **difference-based** and **ratio-based** metrics for **regression errors** instead of classification outcomes.



In [ ]:
# Définition des groupes privilégiés et non privilégiés, en cohérence avec df_clean
priv = [{'sex': 1}]   # Male
unpriv = [{'sex': 0}] # Female

# Création de BinaryLabelDatasetMetric pour le dataset original `dataset_aif`
# (dataset_aif a été créé dans la cellule Y9b-yY2rH4x5)
metric_orig = BinaryLabelDatasetMetric(dataset_aif, privileged_groups=priv, unprivileged_groups=unpriv)

# Calcul et affichage des métriques
spd = metric_orig.statistical_parity_difference()
di = metric_orig.disparate_impact()

print("--- Métriques d'Équité (Dataset Original via dataset_aif) ---")
print(f"Statistical Parity Difference (SPD) : {spd:.4f}")
print(f"Disparate Impact (DI)               : {di:.4f}")


--- Métriques d'Équité (Dataset Original via dataset_aif) ---
Statistical Parity Difference (SPD) : -0.1989
Disparate Impact (DI)               : 0.3635


## 3. Create a fairer dataset with Reweighing

Use AIF360's **Reweighing** preprocessing algorithm to compute instance weights that compensate for bias between the protected groups.

Steps:

1. Instantiate a `Reweighing` object with the same `privileged_groups` and `unprivileged_groups`.  
2. Call `fit_transform` on the original dataset to obtain a transformed dataset `adult_rw`.  
3. Recompute the fairness metrics on the reweighted dataset and compare to the original.



In [ ]:
# Instanciation de Reweighing et transformation du dataset
# On utilise le `dataset_aif` existant pour la transformation
rw = Reweighing(unprivileged_groups=unpriv, privileged_groups=priv)
adult_rw = rw.fit_transform(dataset_aif)

# Calcul des métriques sur le dataset reweighté
metric_rw = BinaryLabelDatasetMetric(adult_rw, privileged_groups=priv, unprivileged_groups=unpriv)

# Les métriques seront affichées dans la cellule suivante (Tlt-_gT0EWD9)


In [ ]:
# Les métriques originales (spd, di) ont été calculées dans la cellule précédente (xK4v-tteCZ53 ou m37-MQR4kLrE).
# On les récupère pour l'affichage comparatif.
# Si spd et di ne sont pas définis ici, il faut s'assurer que la cellule les définissant a été exécutée.
# Pour la robustesse, on peut les recalculer ici si nécessaire, mais l'idée est d'utiliser les valeurs de l'étape précédente.
# Assuming spd and di from previous execution are available in global scope.

# 4. Affichage et comparaison
print("--- MESURES DE BIAIS (APRÈS REWEIGHING) ---")
print(f"New Statistical Parity Difference (SPD) : {metric_rw.statistical_parity_difference():.4f}")
print(f"New Disparate Impact (DI)               : {metric_rw.disparate_impact():.4f}")

spd_rw = metric_rw.statistical_parity_difference()
di_rw = metric_rw.disparate_impact()

print("--- Métriques d'Équité (Après Reweighing) ---")
print(f"Statistical Parity Difference (SPD) : {spd_rw:.4f}")
print(f"Disparate Impact (DI)               : {di_rw:.4f}")

# Petit bonus pour voir l'évolution
print("\nÉvolution :")
# On suppose que `spd` et `di` ont été calculés et sont disponibles dans l'environnement.
# Si ce n'est pas le cas, vous pourriez avoir une erreur NameError. Il est recommandé
# de lancer les cellules dans l'ordre pour que ces variables soient définies.
print(f"SPD: {spd:.4f} -> {spd_rw:.4f}")
print(f"DI : {di:.4f} -> {di_rw:.4f}")


--- MESURES DE BIAIS (APRÈS REWEIGHING) ---
New Statistical Parity Difference (SPD) : 0.0000
New Disparate Impact (DI)               : 1.0000
--- Métriques d'Équité (Après Reweighing) ---
Statistical Parity Difference (SPD) : 0.0000
Disparate Impact (DI)               : 1.0000

Évolution :
SPD: -0.1989 -> 0.0000
DI : 0.3635 -> 1.0000


## 4. Fair sampling with SQL

We now imagine that we want to create a **training sample** from the Adult dataset for a model that predicts eligibility for a smart-city energy programme.

Our goal is to **avoid sampling bias** in the training data.

1. Convert the original DataFrame to an in-memory SQLite table.  
2. Implement a **balanced sampling** query that selects the same number of rows for each value of `sex`.  
   - Hint: use a window function `ROW_NUMBER() OVER (PARTITION BY sex ORDER BY RANDOM())`.  
3. Bring the sampled data back into pandas and convert it to a `BinaryLabelDataset`.  
4. Compare fairness metrics before and after balanced sampling.



In [ ]:
import sqlite3
import pandas as pd

# 1. Connexion et chargement
conn = sqlite3.connect(":memory:")
# Use df_clean which has 'sex' and 'income' already numerically encoded
df_clean.to_sql("adult", conn, index=False, if_exists="replace")

# 2. Requête SQL (Ton échantillonnage équilibré)
balanced_sql = """
WITH RankedData AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY sex ORDER BY RANDOM()) as row_num
    FROM adult
),
MinCount AS (
    SELECT MIN(cnt) as min_size
    FROM (SELECT COUNT(*) as cnt FROM adult GROUP BY sex)
)
SELECT * FROM RankedData
WHERE row_num <= (SELECT min_size FROM MinCount)
"""

sample_df = pd.read_sql_query(balanced_sql, conn).drop(columns=['row_num'])

# The sample_df now has numerical 'sex' and 'income', but other categorical
# columns are still strings. AIF360's BinaryLabelDataset requires all features
# to be numerical.
# Apply get_dummies to convert remaining categorical columns to numerical.
# This will NOT affect 'income' or 'sex' as they are already numerical.
sample_df_numeric = pd.get_dummies(sample_df)

# 3. Conversion en BinaryLabelDataset (AIF360)
sample_ds = BinaryLabelDataset(
    df=sample_df_numeric,
    label_names=['income'],
    favorable_label=1.0,
    unfavorable_label=0.0,
    protected_attribute_names=['sex']
)

# 4. Calcul des métriques
# 'unpriv' and 'priv' are defined in a previous cell.
metric_balanced = BinaryLabelDatasetMetric(
    sample_ds,
    unprivileged_groups=unpriv,
    privileged_groups=priv
)

print("\n--- Métriques après Échantillonnage SQL Équilibré ---")
print(f"Statistical Parity Difference : {metric_balanced.statistical_parity_difference():.4f}")
print(f"Disparate Impact               : {metric_balanced.disparate_impact():.4f}")


--- Métriques après Échantillonnage SQL Équilibré ---
Statistical Parity Difference : -0.2073
Disparate Impact               : 0.3539


## 5. (Optional) Compare with reweighted dataset

For advanced practice, you can:

- Use the instance weights from the reweighted dataset to guide sampling (approximate **probability-proportional-to-weight** sampling in SQL).  
- Train a simple classifier (e.g., logistic regression) on:
  - the original dataset,
  - the balanced SQL sample, and
  - the reweighted dataset,
  and compare fairness + accuracy.



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Fonction utilitaire pour préparer X (features) et y (label)
def get_X_y(dataset):
    X = dataset.features
    y = dataset.labels.ravel()
    w = dataset.instance_weights
    return X, y, w

# Extraction des données pour les trois cas
X_orig, y_orig, _ = get_X_y(dataset_aif) # Changed adult_ds to dataset_aif
X_rw, y_rw, w_rw = get_X_y(adult_rw)
X_sql, y_sql, _ = get_X_y(sample_ds)

# 1. Modèle Original (Biaisé)
clf_orig = LogisticRegression(max_iter=1000).fit(X_orig, y_orig)

# 2. Modèle Reweighted (Utilise les poids d'instance !)
clf_rw = LogisticRegression(max_iter=1000).fit(X_rw, y_rw, sample_weight=w_rw)

# 3. Modèle SQL Balanced
clf_sql = LogisticRegression(max_iter=1000).fit(X_sql, y_sql)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
results = []

# Nous testons tous les modèles sur les données originales pour voir comment ils prédisent
for name, model in [("Original", clf_orig), ("Reweighted", clf_rw), ("SQL Balanced", clf_sql)]:
    # Prédictions
    preds = model.predict(X_orig)
    acc = accuracy_score(y_orig, preds)

    # Création d'un dataset de prédictions pour calculer les métriques d'équité
    pred_ds = dataset_aif.copy()
    pred_ds.labels = preds.reshape(-1, 1)

    metric = BinaryLabelDatasetMetric(pred_ds, unprivileged_groups=unpriv, privileged_groups=priv)

    results.append({
        "Approche": name,
        "Accuracy": round(acc, 4),
        "Disparate Impact (Pred)": round(metric.disparate_impact(), 4),
        "Stat. Parity Diff (Pred)": round(metric.statistical_parity_difference(), 4)
    })

# Affichage des résultats
import pandas as pd
df_res = pd.DataFrame(results)
print(df_res)

       Approche  Accuracy  Disparate Impact (Pred)  Stat. Parity Diff (Pred)
0      Original    0.8389                   0.3001                   -0.1754
1    Reweighted    0.8301                   0.6984                   -0.0619
2  SQL Balanced    0.8374                   0.3042                   -0.1763
